# For the venues, fetches data, creates and queries graph database (is being tested).

# important!: open neo4j, gather required libraries

In [ ]:
    !pip install meetup.api
    import meetup.api
    import json

    import pandas
    from pandas.io.json import json_normalize

    
    client = meetup.api.Client('149155b67979442634b463d11cc')
    
    range

In [59]:
    # creation of the CSV files for Groups, Events and RSVPs
    def createVenuesCSV(venuesRange, nameOfVenuesRange):
        fileVenues = "venues" + nameOfVenuesRange + ".csv"

        f = open(fileVenues, "w+")
        f.close()
        
        #create Venues-CSV
        venueDF = pandas.DataFrame.from_dict(venuesRange.__dict__["results"])
        venueDF = venueDF[['id','rating_count', 'rating', 'address_1','city', 'localized_country_name', 'lon', 'lat']]
        #venueDF = venueDF.to_json(orient='records')
       
        venueDF.to_csv(fileVenues, mode='a', encoding='utf-8')

In [ ]:
ZH = client.GetOpenVenues(country = "CH", city="Zürich", radius=150)
createVenuesCSV(ZH, "ZH")

# graph creation

In [64]:
import os
import csv
from neo4j import GraphDatabase, basic_auth

#currently neo4j is running on localhost
neo4jUrl = os.environ.get('NEO4J_URL',"bolt://localhost")
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"1")

driver = GraphDatabase.driver(neo4jUrl, auth=basic_auth(neo4jUser, neo4jPass))

graph = driver.session()

In [65]:
# create graph databases for groups, events, rsvps and the 
# relationships between them using the specified csv files
def createGraphUsingCSVV(venuesRange):

        fileVenues = "venues" + venuesRange + ".csv"


        with open(fileVenues, encoding='utf8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    id = row['id']
                    rating_count = row['rating_count']
                    rating = row['rating']
                    city = row['city']
                    address_1 = row['address_1']
                    localized_country_name = row['localized_country_name']
                    lon = row['lon']
                    lat = row['lat']
                    graph.run ("MERGE (v:Venue {id: {id}, rating_count: {rating_count}, rating : {rating}, city : {city}, address_1 : {address_1}, localized_country_name:{localized_country_name}, lon : {lon}, lat : {lat}})"
                              , id = id, city=city, rating_count = rating_count, rating = rating, address_1 = address_1, localized_country_name = localized_country_name, lon = lon, lat = lat
                              )

       

In [66]:
createGraphUsingCSVV("ZH")

graph.close()

# querying

In [ ]:
# get top 5 events locations based on amount of events in a specific place(from meetup's openVenues-API; time frame not given)

In [ ]:
def getHotEventsVenue(tx):
    for record in tx.run("match (e:Venue) where not e.address_1 = '' and  not e.city='' return e.address_1 as address, e.city as city, count(*) as amountOfEvents order by  amountOfEvents desc limit 5"):
        print("AmountOfEvents "+str(record["amountOfEvents"]) + " | "+str(record["address"]) + " | " +str(record["city"]))

with driver.session() as session:
    session.read_transaction(getHotEventsVenue)  

In [ ]:
# get top 5 event locations based on rating in a specific place(from meetup's openVenues-API; time frame not given)

In [ ]:
def getHotVenuesBasedOnRating(tx):
    for record in tx.run("match (e:Venue) where not e.address_1 = '' and  not e.city='' and toInt(e.rating_count) > 10 return e.address_1 as address, e.city as city, e.rating as rating, e.rating_count as rating_count order by toFloat(rating) desc limit 5"):
        print(str(record["address"]) + " | "+str(record["rating"]) + " | " +str(record["rating_count"]) )

with driver.session() as session:
    session.read_transaction(getHotVenuesBasedOnRating)  